In [1]:
!pip install memory_profiler
!pip install polars==1.6.0
!pip install --extra-index-url=https://pypi.nvidia.com polars[gpu]
!pip install pip install cudf-polars-cu12
!pip install duckdb
!pip install chdb
!pip install fastexcel
!pip install altair

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [2]:
import polars as pl 
import duckdb
import chdb
import pandas as pd
import altair as alt

In [3]:
%load_ext memory_profiler
pl.Config.set_fmt_float("full")

polars.config.Config

# chDB In-Memory

In [4]:
%%timeit -r 5 -n 2
%%memit
chdb_df = pl.from_pandas(chdb.query(
    """
        SELECT COUNTRY_CODE, PROJECT, COUNT(*) AS DOWNLOADS FROM file("/kaggle/input/pypi-parquet/pypi.parquet", Parquet)
        GROUP BY COUNTRY_CODE, PROJECT
        ORDER BY DOWNLOADS DESC;
    """,
    "Dataframe"
))

peak memory: 846.92 MiB, increment: 415.41 MiB
peak memory: 841.12 MiB, increment: 58.84 MiB
peak memory: 888.04 MiB, increment: 120.64 MiB
peak memory: 927.72 MiB, increment: 111.48 MiB
peak memory: 957.68 MiB, increment: 116.37 MiB
peak memory: 970.60 MiB, increment: 93.76 MiB
peak memory: 996.25 MiB, increment: 149.94 MiB
peak memory: 1016.90 MiB, increment: 145.79 MiB
peak memory: 1029.79 MiB, increment: 142.96 MiB
peak memory: 1049.18 MiB, increment: 115.37 MiB
4.61 s ± 48.6 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


# chDB MergeTree

In [5]:
from chdb.session import Session
s = Session(path="/kaggle/working/pypi_dataset")

In [6]:
%%timeit -r 1 -n 1
%%memit

s.query("CREATE DATABASE IF NOT EXISTS pypi")
s.query(
    """
    CREATE TABLE IF NOT EXISTS pypi.pypi
    ENGINE = MergeTree()
    ORDER BY ()
    AS
    SELECT * 
    FROM file('/kaggle/input/pypi-parquet/pypi.parquet');
    """
)

peak memory: 1065.43 MiB, increment: 96.37 MiB
925 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%%timeit -r 5 -n 2
%%memit

chdb_disk_df = s.query(
    """
    SELECT COUNTRY_CODE, PROJECT, COUNT(*) AS DOWNLOADS FROM pypi.pypi
    GROUP BY COUNTRY_CODE, PROJECT
    ORDER BY DOWNLOADS DESC;
    """,
    "Dataframe"
)

peak memory: 1124.00 MiB, increment: 61.93 MiB
peak memory: 1105.92 MiB, increment: -11.29 MiB
peak memory: 1113.10 MiB, increment: 39.98 MiB
peak memory: 1133.14 MiB, increment: 45.68 MiB
peak memory: 1169.33 MiB, increment: 54.79 MiB
peak memory: 1170.05 MiB, increment: 23.04 MiB
peak memory: 1200.41 MiB, increment: 46.73 MiB
peak memory: 1218.96 MiB, increment: 52.96 MiB
peak memory: 1225.59 MiB, increment: 24.41 MiB
peak memory: 1254.56 MiB, increment: 33.77 MiB
2.71 s ± 17.7 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


# DuckDB In-Memory

In [8]:
%%timeit -r 5 -n 2
%%memit

conn = duckdb.connect(database=':memory:')
conn.execute(
    """
        SET enable_progress_bar = true;
    """
)    
duckdb_mem_df = conn.execute(
    """
        SELECT COUNTRY_CODE, PROJECT, COUNT(*) AS DOWNLOADS FROM '/kaggle/input/pypi-parquet/pypi.parquet'
        GROUP BY COUNTRY_CODE, PROJECT
        ORDER BY DOWNLOADS DESC;
    """
).pl()

peak memory: 1296.05 MiB, increment: 57.45 MiB
peak memory: 1281.95 MiB, increment: 0.00 MiB
peak memory: 1231.98 MiB, increment: 0.00 MiB
peak memory: 1225.76 MiB, increment: 0.08 MiB
peak memory: 1227.21 MiB, increment: 1.45 MiB
peak memory: 1225.73 MiB, increment: -1.48 MiB
peak memory: 1228.58 MiB, increment: 5.83 MiB
peak memory: 1228.58 MiB, increment: 0.00 MiB
peak memory: 1227.71 MiB, increment: 1.68 MiB
peak memory: 1228.48 MiB, increment: 0.77 MiB
1.76 s ± 76.4 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


# DuckDB Persistent

In [9]:
%%timeit -r 1 -n 1
%%memit
conn = duckdb.connect(database='/kaggle/working/duckdb.db')
conn.execute(
    """
        SET enable_progress_bar = true;
    """
)
conn.execute(
    """
        CREATE TABLE IF NOT EXISTS pypi AS
            SELECT * FROM '/kaggle/input/pypi-parquet/pypi.parquet'
    """
)
conn.execute(
    """
        CREATE INDEX IF NOT EXISTS ctrycode_prjt ON pypi (COUNTRY_CODE, PROJECT);
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

peak memory: 1797.82 MiB, increment: 569.34 MiB
41 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -r 5 -n 2
%%memit
duckdb_disk_df = conn.execute(
    """
        SELECT COUNTRY_CODE, PROJECT, COUNT(*) AS DOWNLOADS FROM pypi
        GROUP BY COUNTRY_CODE, PROJECT
        ORDER BY DOWNLOADS DESC;
    """
).pl()

peak memory: 1850.24 MiB, increment: 56.64 MiB
peak memory: 1877.44 MiB, increment: 27.20 MiB
peak memory: 1879.39 MiB, increment: 1.95 MiB
peak memory: 1879.19 MiB, increment: 14.01 MiB
peak memory: 1883.64 MiB, increment: 4.45 MiB
peak memory: 1883.69 MiB, increment: 14.41 MiB
peak memory: 1883.98 MiB, increment: 0.29 MiB
peak memory: 1869.73 MiB, increment: 0.11 MiB
peak memory: 1799.90 MiB, increment: 11.19 MiB
peak memory: 1801.55 MiB, increment: 15.02 MiB
1.4 s ± 10.2 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


# Polars

In [11]:
%%timeit -r 1 -n 1
%%memit

pypi_df = pl.scan_parquet('/kaggle/input/pypi-parquet/pypi.parquet')
query = (
    pypi_df
    .group_by(["COUNTRY_CODE", "PROJECT"])  # This is correct for LazyFrame
    .agg(pl.len().alias("DOWNLOADS"))    # Count rows
    .sort("DOWNLOADS", descending=True)    # Sort by DOWNLOADS in descending order
)


peak memory: 1803.06 MiB, increment: 1.50 MiB
283 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Polars CPU

In [12]:
%%timeit -r 5 -n 2
%%memit
pl_cpu_df = query.collect()

peak memory: 5335.65 MiB, increment: 3532.58 MiB
peak memory: 5615.59 MiB, increment: 2700.91 MiB
peak memory: 5668.55 MiB, increment: 2696.00 MiB
peak memory: 5677.63 MiB, increment: 2695.12 MiB
peak memory: 5682.53 MiB, increment: 2695.45 MiB
peak memory: 5697.02 MiB, increment: 2696.11 MiB
peak memory: 5706.31 MiB, increment: 2694.97 MiB
peak memory: 5710.00 MiB, increment: 2695.07 MiB
peak memory: 5711.19 MiB, increment: 2695.20 MiB
peak memory: 5712.09 MiB, increment: 2695.38 MiB
5.64 s ± 205 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


## Polars GPU

In [13]:
%%timeit -r 5 -n 2
%%memit
pl_gpu_df = query.collect(engine="gpu")

peak memory: 3622.05 MiB, increment: 604.97 MiB
peak memory: 3657.61 MiB, increment: 35.16 MiB
peak memory: 3666.80 MiB, increment: 30.63 MiB
peak memory: 3655.09 MiB, increment: 25.31 MiB
peak memory: 3666.80 MiB, increment: 30.63 MiB
peak memory: 3664.10 MiB, increment: 27.91 MiB
peak memory: 3655.04 MiB, increment: 25.30 MiB
peak memory: 3666.74 MiB, increment: 30.60 MiB
peak memory: 3666.73 MiB, increment: 30.63 MiB
peak memory: 3666.74 MiB, increment: 30.62 MiB
1.19 s ± 553 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


# Pandas

In [14]:
%%timeit -r 5 -n2
%%memit

pypi_df = pd.read_parquet('/kaggle/input/pypi-parquet/pypi.parquet')
pd_df = (
    pypi_df
    .groupby(["COUNTRY_CODE", "PROJECT"])
    .size()
    .reset_index(name="DOWNLOADS")
    .sort_values("DOWNLOADS", ascending=False)
)

peak memory: 24784.81 MiB, increment: 21148.67 MiB
peak memory: 27702.67 MiB, increment: 20057.39 MiB
peak memory: 27965.23 MiB, increment: 21576.84 MiB
peak memory: 27841.43 MiB, increment: 21404.06 MiB
peak memory: 27778.88 MiB, increment: 21530.22 MiB
peak memory: 27446.39 MiB, increment: 21291.94 MiB
peak memory: 27752.27 MiB, increment: 21395.42 MiB
peak memory: 27797.41 MiB, increment: 21575.43 MiB
peak memory: 27705.09 MiB, increment: 21533.64 MiB
peak memory: 27316.38 MiB, increment: 21096.46 MiB
50.9 s ± 1.74 s per loop (mean ± std. dev. of 5 runs, 2 loops each)


# Resource Utilization

In [15]:
df = pl.read_excel('/kaggle/input/resource-utilization/memory_usage_analysis.xlsx')
avg_df = df.group_by('Technology').agg([
    pl.col('Peak Memory (MiB)').mean(),
    pl.col('Increment (MiB)').mean(),
    pl.col('Time per loop (s)').mean()
]).sort('Peak Memory (MiB)', descending=True)

df_pandas = df.to_pandas()
avg_df_pandas = avg_df.to_pandas()

# Create a color scale
color_scale = alt.Scale(domain=['chDB In-Memory', 'chDB Merge Tree', 'DuckDB In-Memory', 'DuckDB Persistent', 'Polars CPU', 'Polars GPU', 'Pandas'],
                        range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2'])

# 1. Bar chart for Peak Memory Usage
peak_memory_chart = alt.Chart(avg_df_pandas).mark_bar().encode(
    x=alt.X('Technology:N', sort='-y'),
    y=alt.Y('Peak Memory (MiB):Q', title='Average Peak Memory (MiB)'),
    color=alt.Color('Technology:N', scale=color_scale),
    tooltip=['Technology', 'Peak Memory (MiB)']
).properties(
    title='Average Peak Memory Usage by Technology',
    width=600,
    height=400
)

# 2. Bar chart for Time per Loop
time_chart = alt.Chart(avg_df_pandas).mark_bar().encode(
    x=alt.X('Technology:N', sort='-y'),
    y=alt.Y('Time per loop (s):Q', title='Average Time per Loop (s)'),
    color=alt.Color('Technology:N', scale=color_scale),
    tooltip=['Technology', 'Time per loop (s)']
).properties(
    title='Average Time per Loop by Technology',
    width=600,
    height=400
)

# Combine all charts
final_chart = (peak_memory_chart & time_chart)

# Display the chart
final_chart.show()



alt.VConcatChart(...)